# Assignment 2

Deadline: 26.03.2025, 12:00 CET

Rakhmatillo Khonkhoshimov, Student ID: 123456, email@example.com

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [7]:
# Import standard libraries
import os
import sys
import types  # Added for MethodType

# Import third-party libraries
import numpy as np
import pandas as pd

# Set up paths
# Assume the notebook is in the assignments folder: qpmwp-course/assignments
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
sys.path.insert(0, src_path)  # Prepend src to the path

# Now import your local modules
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [8]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
    """
    Linearize the turnover constraint in the quadratic programming problem.

    This method modifies the QP to account for a turnover constraint of the form
      sum(|x - x_init|) <= to_budget,
    by introducing auxiliary variables u_i that satisfy:
      u_i >= x_i - x_init_i    and    u_i >= -(x_i - x_init_i).

    The updated decision variable is z = [x, u] ∈ ℝ^(2n). The original QP is updated
    with new matrices/vectors (P, q, G, h, A, lb, ub).

    Parameters:
    -----------
    x_init : np.ndarray
        The initial portfolio weights (vector of length n).
    to_budget : float, optional
        The maximum allowable turnover. Defaults to float('inf').
    """

    # Original dimension (number of assets)
    n = len(x_init)
    
    # Retrieve original problem data
    P_orig = self.problem_data.get('P')
    q_orig = self.problem_data.get('q')
    G_orig = self.problem_data.get('G')
    h_orig = self.problem_data.get('h')
    A_orig = self.problem_data.get('A')
    b_orig = self.problem_data.get('b')
    lb_orig = self.problem_data.get('lb')
    ub_orig = self.problem_data.get('ub')

    # --- Extend the objective ---
    # Our new decision variable z = [x, u] ∈ ℝ^(2n).
    # We assume u has zero cost. Hence, new objective:
    #   0.5 zᵀ P_new z + q_newᵀ z,
    # where
    #   P_new = [ P_orig      0 ]
    #           [ 0      0_(n×n) ]
    # and
    #   q_new = [ q_orig ]
    #           [ 0      ]
    new_P = np.block([
        [P_orig, np.zeros((n, n))],
        [np.zeros((n, n)), np.zeros((n, n))]
    ])
    new_q = np.concatenate([q_orig, np.zeros(n)])

    # --- Extend the inequality constraints ---
    # Start with original inequalities: G_orig x <= h_orig.
    # When we lift the variable to z, we extend these with zeros for u.
    if G_orig is not None:
        m_orig = G_orig.shape[0]
        G1 = np.hstack([G_orig, np.zeros((m_orig, n))])
        h1 = h_orig
    else:
        G1 = np.empty((0, 2*n))
        h1 = np.empty((0,))

    # Next, add constraints for each asset i to capture the absolute value:
    #   (i)   x_i - u_i <= x_init_i     (which enforces u_i >= x_i - x_init_i)
    #   (ii) -x_i - u_i <= -x_init_i      (which enforces u_i >= -(x_i - x_init_i))
    G2 = np.zeros((2*n, 2*n))
    h2 = np.zeros(2*n)
    for i in range(n):
        # Constraint (i): x_i - u_i <= x_init_i
        G2[i, i] = 1        # coefficient for x_i
        G2[i, n + i] = -1   # coefficient for u_i
        h2[i] = x_init[i]
        # Constraint (ii): -x_i - u_i <= -x_init_i
        G2[n + i, i] = -1
        G2[n + i, n + i] = -1
        h2[n + i] = -x_init[i]

    # Finally, add the turnover budget constraint: sum_i u_i <= to_budget.
    G3 = np.zeros((1, 2*n))
    G3[0, n:] = 1  # Only u variables have nonzero coefficients.
    h3 = np.array([to_budget])

    # Combine all inequality constraints:
    new_G = np.vstack([G1, G2, G3])
    new_h = np.concatenate([h1, h2, h3])

    # --- Extend equality constraints ---
    # If any equality constraints exist in the original problem (A_orig x = b_orig),
    # we simply extend them by adding zeros for the u variables.
    if A_orig is not None:
        new_A = np.hstack([A_orig, np.zeros((A_orig.shape[0], n))])
        new_b = b_orig
    else:
        new_A = None
        new_b = None

    # --- Extend bounds ---
    # For the original x variables, we keep lb_orig and ub_orig.
    # For the new u variables, we set lower bounds to 0 and upper bounds to infinity.
    if lb_orig is not None:
        new_lb = np.concatenate([lb_orig, np.zeros(n)])
    else:
        new_lb = None
    if ub_orig is not None:
        new_ub = np.concatenate([ub_orig, np.full(n, np.inf)])
    else:
        new_ub = None

    # --- Update the problem data ---
    self.update_problem_data({
        'P': new_P,
        'q': new_q,
        'G': new_G,
        'h': new_h,
        'A': new_A,
        'b': new_b,
        'lb': new_lb,
        'ub': new_ub
    })

    return None


## Demo

#### Create P and q

In [9]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

# q, P

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [10]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)


### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [11]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

print("Turnover:")
print(np.abs(weights - x_init).sum())

Turnover:
0.49954552248142015


/var/folders/b6/r9gq_shd5r1cg8x06t586cf00000gn/T/ipykernel_6671/292018790.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  h2[i] = x_init[i]
/var/folders/b6/r9gq_shd5r1cg8x06t586cf00000gn/T/ipykernel_6671/292018790.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  h2[n + i] = -x_init[i]
